In [1]:
from visualizer import calibration
import visual

from skimage.io import imread
from skimage.io.collection import alphanumeric_key
from dask import delayed
from glob import glob
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import napari
from napari import Viewer
import os

from skimage.registration import phase_cross_correlation
import cv2 as cv
from skimage import io

import cupy as cp 
import imagej

%gui qt 
%matplotlib inline
plt.style.use('dark_background')


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# main_folder = r'E:\Ed\Data\20220324_drosophila_egg\20220324_M25_head-3_WL'
# main_folder = r'E:\Ed\Data\test_flies\20220322_M25_larvae_05'
main_folder = r'E:\Ed\Data\20220324_drosophila_egg\20220324_M25_PSF1'
# main_folder = r'E:\Ed\Data\20220331_drophila_egg\20220331_M25_egg_WL'
# main_folder = r'E:\Ed\Data\20220331_drophila_egg\20220331_M25_egg1'

# folder = r'E:\Ed\PSF\20220124_M25_allcameras\CAM_Z2'
# file_extension = folder + '/' + '*.raw'
folder_names = sorted(glob(main_folder + '/CAM*/'), key=alphanumeric_key)
offsets_file = os.path.join(main_folder,"offsets.csv")

# file_names = sorted(glob(file_extension),key=alphanumeric_key)

In [3]:
#Scope Parameters
# FOV = 50e-6
cam_px = 5.86e-6
totalmag = 37.5
px_size_img = cam_px/totalmag
zstep = 2e-6
z_scale = zstep/px_size_img

print(px_size_img)
print(zstep/cam_px)
print(z_scale)

1.5626666666666666e-07
0.341296928327645
12.79863481228669


In [4]:
try:
    stack= calibration.load_dataset(main_folder,'uint8')
    print("8bit")
except: 
    print("16bit")
    stack = calibration.load_dataset(main_folder,'uint16')

8bit


In [5]:
# Load offsets from desired folder 
# main_folder = r'E:\Ed\Data\20220324_drosophila_egg\20220324_M25_PSF2-16bit'
main_folder = r'E:\Ed\Data\20220324_drosophila_egg\20220324_M25_PSF1'
offsets_file_load = os.path.join(main_folder,"offsets.csv")
with open(offsets_file_load) as file_name:
    offsets = np.loadtxt(file_name, delimiter=",")

In [4]:
# px_depth = 'uint16'
# # px_depth = 'uint8'

# raw_M25_volume = [calibration.dask_raw_ds(fd,px_depth) for fd in folder_names]
# stack = da.concatenate(raw_M25_volume, axis=1)
# stack_max_projection= np.array([ da.max(stack[:,i,:,:],axis=0) for i in range(stack.shape[1])])


In [39]:
# viewer=napari.Viewer()
viewer.add_image(stack,scale=[z_scale,1,1])

<Image layer 'stack' at 0x1e5252c7430>

### Get Calibration from Bead's Files

In [18]:
# Generate our own shifted datset
with cp.cuda.Device(0):
    # test = stack[:,11:15,:,:]
    test = stack
    t,c,h,w  = test.shape
    np_mip =[]
    for i in range(c):
        cp_stack = cp.array(test[:,i,:,:])
        cp_mip = cp.max(cp_stack, axis=0)
        np_mip.append(cp.asnumpy(cp_mip))
        cp_mip = None
        cp_stack=None
    np_mip = np.array(np_mip)
    np_mip = np.expand_dims(np_mip,axis=0)
    

In [20]:
# Show the maximum intensity projection
viewer = napari.Viewer()
viewer.add_image(np_mip, name='mip')

<Image layer 'mip' at 0x1e3e72a44c0>

In [21]:
#Go to napari and make a square for alignment
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]
print(crop_region)
print(template_box)
print(template_box.shape)


[[  0.          12.         292.48246486 393.88598238]
 [  0.          12.         292.48246486 397.39475431]
 [  0.          12.         295.99123679 397.39475431]
 [  0.          12.         295.99123679 393.88598238]]
[[292 394]
 [296 397]]
(2, 2)


In [22]:
# Since OpenCV Match Template takes in uint8 || float32. We can change from uint16 to uint8 easily
np_mip_8bit =visual.im_bit_convert(np_mip,bit=8)
min_val, max_val = template_box
t,c = template[0,0:2]
crop_img = np_mip_8bit[t,c, min_val[0]:max_val[0],min_val[1]:max_val[1]]
viewer.add_image(crop_img,name='crop')

<Image layer 'crop' at 0x1e3a58b6c10>

In [57]:
np_mip_8bit[0,i,:,:].shape

(600, 960)

In [23]:
coord =[]
w,h = template.shape  
methods = 'cv.TM_CCOEFF'
print(w,h)
t,c,h,w = np_mip_8bit.shape

for i in range(c):
    method = eval(methods) # This method parses expression and runs ()
    # Apply template Matching
    # Slide through image and compare template patches.
    # Comparison of best matches is foudn as global minn in SQDIFF or max in CCORR or CCOEF.
    res = cv.matchTemplate(np_mip_8bit[0,i,:,:],crop_img,method)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
        
    bottom_right = (top_left[0] + w, top_left[1] + h)
    #Output would contain the Top left corner and bottom right in case of need to use this rectangle
    coord.append(((top_left[1],top_left[0]),(bottom_right[1],bottom_right[0])))

#For the bead calibration we only use the TL points for alignment
TL_coords = np.array(coord)
TL_coords= TL_coords[0:25,0]

#Coordinates need to then be subtracted to the reference frame (i.e t,c for which rectangle was drawn)
res = []
ref_cam_index = 0
# diff =[]
for i,coordinate in enumerate(TL_coords):
    val = tuple(map(lambda i, j: i - j, TL_coords[ref_cam_index],coordinate))
    res.append(val)
    #If previous values available we can find teh difference between them
    # diff.append(tuple(map(lambda i, j: i + j, val,shift_stack_coord[i])))
# print(diff)
print(res)
# print(shift_stack_coord)
offset_coordinates = np.array(res)

#Save the Offsets in the folder
np.savetxt(offsets_file, offset_coordinates, delimiter=',')

2 4
[(0, 0), (21, 23), (6, -148), (5, -139), (42, -167), (22, -64), (47, -47), (51, -45), (37, -61), (92, -88), (72, 22), (51, 10), (69, 7), (98, -10), (95, -21), (34, 38), (98, -19), (85, 30), (61, -50), (105, -46), (101, 39), (99, 19), (106, -29), (99, -43), (129, -25)]


In [24]:
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

#Convert to Dask Array
stack_aligned = da.from_array(stack_aligned)


(25, 3)
(385, 25, 600, 960)


In [65]:
# stack_aligned =  np.moveaxis(stack_aligned,0,1)
stack_aligned.shape

(385, 25, 600, 960)

In [40]:
viewer.add_image(stack_aligned, name='test',scale = [z_scale,1,1])

<Image layer 'test' at 0x1e52529b8e0>

### Align Datasets with Known Offsets


In [6]:
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset

offset_coordinates = offsets
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

(25, 3)
(385, 25, 600, 960)


In [7]:
#Convert numpy to dask?
da_stack_align = da.from_array(stack_aligned)
print(da_stack_align)

dask.array<array, shape=(385, 25, 600, 960), dtype=uint8, chunksize=(175, 25, 150, 160), chunktype=numpy.ndarray>


In [29]:
viewer= napari.Viewer()

Exception in callback BaseAsyncIOLoop._handle_events(7100, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(7100, 1)>
Traceback (most recent call last):
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\Callisto\Anaconda3\envs\napa

In [9]:
viewer.add_image(da_stack_align, name='dask_align',scale=[z_scale,1,1])

<Image layer 'dask_align' at 0x18d23372550>

### CROP PSF TO ANALYZE


In [10]:
#Go to napari and make a square for alignment
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]
print(crop_region)
print(template_box)
print(template_box.shape)

[[192.          12.         387.65098341 428.55736046]
 [192.          12.         387.65098341 440.79135307]
 [192.          12.         397.71442895 440.79135307]
 [192.          12.         397.71442895 428.55736046]]
[[388 429]
 [398 441]]
(2, 2)


In [30]:
# Since OpenCV Match Template takes in uint8 || float32. We can change from uint16 to uint8 easily
# np_mip_8bit =visual.im_bit_convert(np_mip,bit=8)
min_val, max_val = template_box
t,c = template[0,0:2]
print(t,c)
crop_img = da_stack_align[:,:, min_val[0]:max_val[0],min_val[1]:max_val[1]]
# crop_img = np_mip_8bit[t,c, min_val[0]:max_val[0],min_val[1]:max_val[1]]
viewer.add_image(crop_img,name='crop')

<string>:1: RuntimeWarning: coroutine 'get_msg' was never awaited


192 12


<Image layer 'crop' at 0x18e4143e460>

In [17]:
crop_img

dask.array<getitem, shape=(385, 25, 10, 12), dtype=uint8, chunksize=(175, 25, 10, 12), chunktype=numpy.ndarray>

### Image J Part

In [34]:
import imagej

def start_imagej():
    global ij
    ij = imagej.init(r'C:\Users\Callisto\Documents\edhirata\software\Fiji.app',headless=False)
    ij.ui().showUI()
    print(ij.getVersion())
    
from PyQt5 import QtCore
QtCore.QTimer.singleShot(0, start_imagej)

In [35]:
def start_imagej():
    import imagej
    global ij
    ij = imagej.init(headless=False)
    ij.ui().showUI()
    print(ij.getVersion())
    
from PyQt5 import QtCore
QtCore.QTimer.singleShot(0, start_imagej)

In [33]:
QtCore.QTimer.singleShot(0, start_imagej)

In [37]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(crop_img,dtype='uint16'))
ij.ui().show('PSF', ij_stack2)

NameError: name 'ij' is not defined